In [6]:
from pyspark.sql import SparkSession
import time
import urllib
import os
# The provided code does not work since I'm on window



# Import findspark and initialize. 
import findspark
findspark.init()

In [7]:
# Import packages

spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [8]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"

In [10]:
spark.sparkContext.addFile(url)
home = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), header=True,inferSchema=True)

In [12]:
# 2. Create a temporary view of the DataFrame.
home.createOrReplaceTempView('home_sales')
home


DataFrame[id: string, date: date, date_built: int, price: int, bedrooms: int, bathrooms: int, sqft_living: int, sqft_lot: int, floors: int, waterfront: int, view: int]

In [14]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?
averagePrice = spark.sql("SELECT date, ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE bedrooms = 4 GROUP BY date ORDER by date")
averagePrice.show()


+----------+-------------+
|      date|average_price|
+----------+-------------+
|2019-01-01|     365088.0|
|2019-01-02|    271644.29|
|2019-01-03|    381218.86|
|2019-01-04|    359586.14|
|2019-01-05|    323683.57|
|2019-01-06|    301091.13|
|2019-01-07|     327474.5|
|2019-01-08|    304567.83|
|2019-01-09|     243619.0|
|2019-01-10|     287844.4|
|2019-01-11|    277574.29|
|2019-01-12|     218507.0|
|2019-01-13|    294603.86|
|2019-01-14|    284434.57|
|2019-01-15|     254288.5|
|2019-01-16|     335106.5|
|2019-01-17|    333439.09|
|2019-01-18|    297542.56|
|2019-01-19|    294484.22|
|2019-01-20|    312395.33|
+----------+-------------+
only showing top 20 rows



In [15]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
averagePrice = spark.sql("SELECT date_built, ROUND(AVG(PRICE), 2) AS average_price FROM home_sales WHERE bedrooms = 2 AND bathrooms = 3 GROUP BY date_built ORDER by date_built")
averagePrice.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    284540.35|
|      2011|    283108.76|
|      2012|    282511.83|
|      2013|    288762.13|
|      2014|    284578.48|
|      2015|    284233.51|
|      2016|    284073.27|
|      2017|    286649.36|
+----------+-------------+



In [16]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?
averagePrice = spark.sql("SELECT date_built, ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE bedrooms = 3 AND bathrooms = 3 AND floors = 2 AND sqft_living >= 2000 GROUP BY date_built ORDER BY date_built")
averagePrice.show()


+----------+-------------+
|date_built|average_price|
+----------+-------------+
|      2010|    285010.22|
|      2011|    276553.81|
|      2012|    307539.97|
|      2013|    303676.79|
|      2014|    298264.72|
|      2015|    297609.97|
|      2016|     293965.1|
|      2017|    280317.58|
+----------+-------------+



In [17]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

averagePrice = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE price >= 350000")
averagePrice.show()



print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|    473796.26|
+-------------+

--- 0.2403583526611328 seconds ---


In [18]:
# 7. Cache the the temporary table home_sales.
spark.sql('cache table home_sales')

DataFrame[]

In [19]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [22]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

averagePrice = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM home_sales GROUP BY view HAVING AVG(price) >= 350000 ORDER BY view desc")
averagePrice.show()

print("--- %s seconds ---" % (time.time() - start_time))


+-------------+
|average_price|
+-------------+
|    1026669.5|
|   1061201.42|
|   1053739.33|
|   1129040.15|
|   1017815.92|
|    1054325.6|
|    1033536.2|
|   1026006.06|
|    970402.55|
|   1137372.73|
|   1062654.16|
|   1107839.15|
|   1031719.35|
|    1072285.2|
|   1070444.25|
|   1056336.74|
|   1117233.13|
|   1033965.93|
|    1063498.0|
|   1053472.79|
+-------------+
only showing top 20 rows

--- 0.8020524978637695 seconds ---


In [23]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home.write.partitionBy('date_built').parquet('parquet_sales',mode='overwrite')

In [24]:
# 11. Read the formatted parquet data.
parquetSales = spark.read.parquet('parquet_sales')

In [26]:
# 12. Create a temporary table for the parquet data.
parquetSales.createOrReplaceTempView('parquet_sales')

In [27]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()

averagePrice = spark.sql("SELECT ROUND(AVG(price), 2) AS average_price FROM home_sales WHERE view >= 350000")
averagePrice.show()




print("--- %s seconds ---" % (time.time() - start_time))

+-------------+
|average_price|
+-------------+
|         NULL|
+-------------+

--- 0.248335599899292 seconds ---


In [28]:
# 14. Uncache the home_sales temporary table.
spark.catalog.uncacheTable("home_sales")

In [30]:
# 15. Check if the home_sales is no longer cached
if spark.catalog.isCached('home_sales'):
    print('home_sales is cached')
else:
    print('home_sales is not cached')


home_sales is not cached
